
<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/fr/b/bb/UQO_Logo.png" alt="UQO" height="120">
  

  <h4 align="center"><b>Technique d'apprentissage[6243] - Enseignant : Allili Mohand Said</b></h4>
  <br>
  <h4 align="center"><b>Projet - IA pour prédire le résultat d'une partie d'échecs</b></h4>
  <h5 align="center"><b>Etudiant - Clément JOURET, Guillaume CAGNIARD</b></h5>
</p>

## IA - Jeu d'échecs

**Objectifs**

Avec ce jeu de données qui contient des informations sur plus de 20 000 parties en ligne d'échecs. Le but est d'utiliser les données du dataset afin de pouvoir prédire le résulat d'une partie (Victoire Blanc, Victoire Noir, Egalité) en fonction d'un ensemble de parmètres.

Dans ce **notebook** on retrouve :
- Exploration de données ✅
- Traitement des donneées ✅
- Entrainement de modèles de classification supervisé ✅
- Evaluation de la perfomance ✅


**Desciption des données :** 

Lien du dataset : https://www.kaggle.com/datasets/datasnaek/chess

L’ensemble des données contient plus de 20 000 instances d'informations sur des parties d'échecs sur le site Lichess.org . Chaque parties est décrite par 16 attributs non numérique qui sont des informations sur le déroulement d'une partie d'échecs. Les attributs qui vont nous intérresé sont :

*  La valeur de l'élo des deux joueurs  
*  La liste des coups
*  L'ouverture qui a été choisie
*  La colone cile qui est le résultat d'une partie

**Implantation du modèle**

Nous allons comparer la perfomance de différents mdoèles en utilisant plusieurs algorithme soit avec une validation croisée en $m=5$ plis ou le partage des données d'entraînement et de validation avec un ration de 0.2.

*   Classificateur de Bayes
*   Régression logistique
*   Réseau de neurones (MLP)

<br><br>


### Import des librairies

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import chess
import chess.engine

### 1 - Exploration des données

In [4]:
# Lecture des données
data = pd.read_csv("games.csv")


df = data.copy() # copie des données
# Infos sur les attributs
df.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [5]:
# Info sur la taille de la matrice des données
df.shape

(20058, 16)

In [6]:
# Info sur les types de données
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  object 
 15  opening_ply     20058 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 2.3+ MB


In [7]:
# Statistiques sur les données, 
df.describe()

,created_at,last_move_at,turns,white_rating,black_rating,opening_ply
count,2.005800e+04,2.005800e+04,20058.000000,20058.000000,20058.000000,20058.000000
mean,1.483617e+12,1.483618e+12,60.465999,1596.631868,1588.831987,4.816981
std,2.850151e+10,2.850140e+10,33.570585,291.253376,291.036126,2.797152
min,1.376772e+12,1.376772e+12,1.000000,784.000000,789.000000,1.000000
25%,1.477548e+12,1.477548e+12,37.000000,1398.000000,1391.000000,3.000000
50%,1.496010e+12,1.496010e+12,55.000000,1567.000000,1562.000000,4.000000
75%,1.503170e+12,1.503170e+12,79.000000,1793.000000,1784.000000,6.000000
max,1.504493e+12,1.504494e+12,349.000000,2700.000000,2723.000000,28.000000


In [8]:
# Statistiques sur les parties
print("La durée moyenne d'une partie est : {0:.2f}".format(df.describe()["turns"]["mean"]), "coups\n")

mean_rating = (df.describe()["white_rating"]["mean"] + df.describe()["black_rating"]["mean"]) / 2
print("L'élo moyen des joueurs est : {0:.2f}".format(mean_rating), "\n")

win_rate_white = len(df.loc[df["winner"] == "white"]) / len(df)
win_rate_black = len(df.loc[df["winner"] == "black"]) / len(df)
draw_rate = len(df.loc[df["winner"] == "draw"]) / len(df)
print("Taux de victoire pour les blancs : {0:.2f}".format(win_rate_white))
print("Taux de victoire pour les noirs : {0:.2f}".format(win_rate_black))
print("Taux d'égalité :  {0:.2f}".format(draw_rate))

La durée moyenne d'une partie est : 60.47 coups

L'élo moyen des joueurs est : 1592.73 

Taux de victoire pour les blancs : 0.50
Taux de victoire pour les noirs : 0.45
Taux d'égalité :  0.05


In [9]:
# print(df["opening_eco"].unique())

# Trouver une façon de representer les données avec un graphique

### 2 - Traitement des données

In [10]:
X = df.drop(["id", "rated", "last_move_at", "created_at", "turns", "victory_status", "increment_code", "black_id", "white_id", "opening_name", "winner"], axis=1)
print(X.head())

   white_rating  black_rating  \
0          1500          1191   
1          1322          1261   
2          1496          1500   
3          1439          1454   
4          1523          1469   

                                               moves opening_eco  opening_ply  
0  d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...         D10            5  
1  d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...         B00            4  
2  e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...         C20            3  
3  d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...         D02            3  
4  e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...         C41            5  


In [11]:
y = df["winner"]
print(y)

0        white
1        black
2        white
3        white
4        white
         ...  
20053    white
20054    black
20055    white
20056    white
20057    black
Name: winner, Length: 20058, dtype: object


In [36]:
def stockfish_evaluation(board, time_limit = 0.01):
    engine = chess.engine.SimpleEngine.popen_uci("stockfish_15_x64_avx2.exe")
    result = engine.analyse(board, chess.engine.Limit(time=time_limit))
    return result["score"].white()


In [37]:
board = chess.Board()
moves = X["moves"].get(0).split()
for i in range(0, X["opening_ply"].get(0)):
    board.push_san(moves[i])
print(stockfish_evaluation(board))

+18


In [39]:
evaluating_board = []
for i in range(0, len(X)):
    moves = X["moves"].get(i).split()[:X["opening_ply"].get(i)]
    board = chess.Board()
    for j in range(0, X["opening_ply"].get(i)):
        board.push_san(moves[j])
    evaluating_board.append(stockfish_evaluation(board))
print(evaluating_board)


OSError: [WinError 1455] Le fichier de pagination est insuffisant pour terminer cette opération